# A/B 테스트 분석

## 환경 설정

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest

# 한글 폰트 설정
!pip install koreanize_matplotlib -q
import matplotlib.pyplot as plt
import koreanize_matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 15.8 MB/s eta 0:00:00


In [ ]:
# Warnings 제거
import warnings
warnings.filterwarnings('ignore')

## 데이터 불러오기

In [ ]:
data_path = '30%20Day%20Subscription%20Data.csv'
df = pd.read_csv(data_path)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            60 non-null     int64  
 1   Date                  60 non-null     object 
 2   Total Users           60 non-null     int64  
 3   Subscribers           60 non-null     int64  
 4   1-Month Subscribers   60 non-null     int64  
 5   12-Month Subscribers  60 non-null     int64  
 6   Revenue               60 non-null     int64  
 7   Group                 60 non-null     object 
 8   Conversion Rate       60 non-null     float64
 9   1-Month Revenue       60 non-null     int64  
 10  12-Month Revenue      60 non-null     int64  
dtypes: float64(1), int64(8), object(2)
memory usage: 5.3+ KB


In [ ]:
df.head()

,Unnamed: 0,Date,Total Users,Subscribers,1-Month Subscribers,12-Month Subscribers,Revenue,Group,Conversion Rate,1-Month Revenue,12-Month Revenue
0,0,2023.9.15,459,22,19,3,1547,A,0.047930,665,882
1,1,2023.9.16,504,26,21,5,2205,A,0.051587,735,1470
2,2,2023.9.17,514,26,22,4,1946,A,0.050584,770,1176
3,3,2023.9.18,482,23,19,4,1841,A,0.047718,665,1176
4,4,2023.9.19,512,25,19,6,2429,A,0.048828,665,1764


In [ ]:
df = df.drop('Unnamed: 0', axis = 1)

In [ ]:
df.head()

,Date,Total Users,Subscribers,1-Month Subscribers,12-Month Subscribers,Revenue,Group,Conversion Rate,1-Month Revenue,12-Month Revenue
0,2023.9.15,459,22,19,3,1547,A,0.047930,665,882
1,2023.9.16,504,26,21,5,2205,A,0.051587,735,1470
2,2023.9.17,514,26,22,4,1946,A,0.050584,770,1176
3,2023.9.18,482,23,19,4,1841,A,0.047718,665,1176
4,2023.9.19,512,25,19,6,2429,A,0.048828,665,1764


## EDA

### A그룹과 B그룹으로 데이터 프레임 나누기

In [ ]:
df_A = df[df['Group'] == 'A']

In [ ]:
df_A = df_A.reset_index(drop=True)

In [ ]:
df_A.head()

,Date,Total Users,Subscribers,1-Month Subscribers,12-Month Subscribers,Revenue,Group,Conversion Rate,1-Month Revenue,12-Month Revenue
0,2023.9.15,459,22,19,3,1547,A,0.047930,665,882
1,2023.9.16,504,26,21,5,2205,A,0.051587,735,1470
2,2023.9.17,514,26,22,4,1946,A,0.050584,770,1176
3,2023.9.18,482,23,19,4,1841,A,0.047718,665,1176
4,2023.9.19,512,25,19,6,2429,A,0.048828,665,1764


In [ ]:
df_B = df[df['Group'] == 'B']

In [ ]:
df_B = df_B.reset_index(drop=True)

In [ ]:
df_B.head()

,Date,Total Users,Subscribers,1-Month Subscribers,12-Month Subscribers,Revenue,Group,Conversion Rate,1-Month Revenue,12-Month Revenue
0,2023.9.15,516,35,28,7,3038,B,0.067829,980,2058
1,2023.9.16,483,33,24,9,3486,B,0.068323,840,2646
2,2023.9.17,488,31,24,7,2898,B,0.063525,840,2058
3,2023.9.18,520,39,31,8,3437,B,0.075000,1085,2352
4,2023.9.19,498,34,30,4,2226,B,0.068273,1050,1176


### A그룹, B그룹 비교분석

- A안 결과
    - A안(기존안)에서는 1개월 구독 비율이 83%, 12개월 구독 비율이 17%였습니다.

In [ ]:
# A그룹(결과 확인겸 코드 확인용)
month_subscribe_ratio_A = df_A['1-Month Subscribers'].sum() / df_A['Subscribers'].sum()
year_subscribe_ratio_A = df_A['12-Month Subscribers'].sum() / df_A['Subscribers'].sum()
print('1개월 구독자 비율(%): {}, 12개월 구독자 비율(%): {}'.format(month_subscribe_ratio_A * 100, year_subscribe_ratio_A * 100))

1개월 구독자 비율(%): 82.77254374158815, 12개월 구독자 비율(%): 17.227456258411845


In [ ]:
# B그룹
month_subscribe_ratio_B = df_B['1-Month Subscribers'].sum() / df_B['Subscribers'].sum()
year_subscribe_ratio_B = df_B['12-Month Subscribers'].sum() / df_B['Subscribers'].sum()
print('1개월 구독자 비율(%): {}, 12개월 구독자 비율(%): {}'.format(month_subscribe_ratio_B * 100, year_subscribe_ratio_B * 100))

1개월 구독자 비율(%): 80.61716489874638, 12개월 구독자 비율(%): 19.382835101253615


In [ ]:
# A그룹 총 구독 전환율
subscribe_total_A = df_A['Subscribers'].sum() / df_A['Total Users'].sum()
print('A그룹 총 구독 전환율: {}'.format(subscribe_total_A))

A그룹 총 구독 전환율: 0.04867343596462496


In [ ]:
# B그룹 총 구독 전환율
subscribe_total_B = df_B['Subscribers'].sum() / df_B['Total Users'].sum()
print('B그룹 총 구독 전환율: {}'.format(subscribe_total_B))

B그룹 총 구독 전환율: 0.06830457120274008


- A그룹 결과
    - 1개월 구독자 비율(%): 82.77
    - 12개월 구독자 비율(%): 17.23
    - 총 구독 전환율(%): 0.049

- B그룹 결과
    - 1개월 구독자 비율(%): 80.62
    - 12개월 구독자 비율(%): 19.38
    - 총 구독 전환율(%): 0.068

In [ ]:
#총 수익 비교수익
total_revenue_A = df_A['Revenue'].sum()
total_revenue_B = df_B['Revenue'].sum()
print('A그룹 총 수익: {} \nB그룹 총 수익: {}'.format(total_revenue_A, total_revenue_B))

A그룹 총 수익: 59157 
B그룹 총 수익: 88354


In [ ]:
# 실제 바뀐 이미지의 영향이 있었는지 확인하기위해 수익 중 12개월 구독에 의한 수익 및 비율 확인
revenue_year_A = df_A['12-Month Revenue'].sum()
revenue_year_B = df_B['12-Month Revenue'].sum()
revenue_year_ratio_A = df_A['12-Month Revenue'].sum() / df_A['Revenue'].sum()
revenue_year_ratio_B = df_B['12-Month Revenue'].sum() / df_B['Revenue'].sum()
print('A그룹 12개월 구독자 수익: {} \nB그룹 12개월 구독자 수익: {}'.format(revenue_year_A, revenue_year_B))
print('A그룹 총 수익 대비 12개월 구독자 수익 비율: {} \nB그룹 총 수익 대비 12개월 구독자 수익 비율: {}'.format(revenue_year_ratio_A, revenue_year_ratio_B))

A그룹 12개월 구독자 수익: 37632 
B그룹 12개월 구독자 수익: 59094
A그룹 총 수익 대비 12개월 구독자 수익 비율: 0.6361377351792687 
B그룹 총 수익 대비 12개월 구독자 수익 비율: 0.6688321977499604


## 결과

- A그룹 결과
    - 1개월 구독자 비율(%): 82.77
    - 12개월 구독자 비율(%): 17.23
    - 총 구독 전환율(%): 0.049
    - A그룹 총 수익: 59157
    - A그룹 12개월 구독자 수익: 37632
    - A그룹 총 수익 대비 12개월 구독자 수익 비율(%): 0.6361377351792687

- B그룹 결과
    - 1개월 구독자 비율(%): 80.62
    - 12개월 구독자 비율(%): 19.38
    - 총 구독 전환율(%): 0.068
    - B그룹 총 수익: 88354
    - B그룹 12개월 구독자 수익: 59094
    - B그룹 총 수익 대비 12개월 구독자 수익 비율(%): 0.6688321977499604


위 결과를 통해 B그룹의 총 구독전환율, 수익, 12개월 구독자 수익의 비율 등에서 B그룹이 더 높게 나타났음을 확인할 수 있다.

### 통계적 분석

In [ ]:
# 검정 진행
total_users_AB = [df_A['Total Users'].sum(), df_B['Total Users'].sum()]
subscribers_year_AB = [df_A['12-Month Subscribers'].sum(), df_B['12-Month Subscribers'].sum()]

In [ ]:
# Z-test
z_stats, p_value = proportions_ztest(subscribers_year_AB, total_users_AB, alternative='two-sided')

In [ ]:
print(f'Z-stat: {z_stats:.3f}')
print(f'p_value:{p_value:.5f}')

Z-stat: -4.096
p_value:0.00004


- p값이 유의미 함으로 두 그룹 전환율의 차이는 통계적으로 유의미함.

B그룹(개선안)에 대하여 긍정적인 결과와 통계적으로 유의미한 결과임을 입증하였기 때문에 개선안 변경을 추진하는것이 좋은 방향이라는 결론을 도출함